In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def LIFez(I):
    """
    Runs a LIF simulation on neuron and returns outputted voltage

            Parameters:
                    I (double[]): mV
                    Cm (double): The membrane capacitance
                    Rm (double): The membrane resistance

            Returns:
                    V (double[]): A list of the output voltages in mV
    """
    V_thresh = 30
    V_rest = -65
    V_spike = 80
    Cm=4
    Rm=5
    
    # an array of time
    T = 100                      
    dT  = 0.02                   
    time = np.arange(0, T+dT, dT) 
    I = np.ones(len(time))*I

    # default voltage list set to resting volatage of -65mV
    V = (-65) * np.ones(len(time))
    #V_bin = 0*np.ones(len(time))

    did_spike = False

    # function member variable to track spikes
    spikes = 0

    for t in range(len(time)):
        # using "I - V(t)/Rm = Cm * dV/dT"
        dV = (I[t] - (V[t - 1] - V_rest) / Rm) / Cm

        # reset membrane potential if neuron spiked last tick
        if did_spike:
            V[t] = V_rest + dV * dT
        else:
            V[t] = V[t - 1] + dV * dT

        # check if membrane voltage exceeded threshold (spike)
        if V[t] > V_thresh:
            did_spike = True
            #V_bin[t] = 80
            # set the last step to spike value
            V[t] = V_spike
            spikes += 1
        else:
            did_spike = False
    spike_rate=spikes/100*100
    return V,spike_rate

In [10]:
def LIF(I):
    """
    Runs a LIF simulation on neuron and returns outputted voltage

            Parameters:
                    I (double[]): A list of input voltages in mV
                    Cm (double): The membrane capacitance
                    Rm (double): The membrane resistance

            Returns:
                    V (double[]): A list of the output voltages in mV
    """
    V_thresh = 30
    V_rest = -65
    V_spike = 80
    dT = 0.02  # time step in ms
    total_time = (I.size) * dT
    Cm=4
    Rm=5
    
    # an array of time
    time = np.arange(0, total_time, dT)

    # default voltage list set to resting volatage of -65mV
    V = (-65) * np.ones(len(time))
    #V_bin = 0*np.ones(len(time))

    did_spike = False

    # function member variable to track spikes
    spikes = 0

    for t in range(len(time)):
        # using "I - V(t)/Rm = Cm * dV/dT"
        dV = (I[t] - (V[t - 1] - V_rest) / Rm) / Cm

        # reset membrane potential if neuron spiked last tick
        if did_spike:
            V[t] = V_rest + dV * dT
        else:
            V[t] = V[t - 1] + dV * dT

        # check if membrane voltage exceeded threshold (spike)
        if V[t] > V_thresh:
            did_spike = True
            #V_bin[t] = 80
            # set the last step to spike value
            V[t] = V_spike
            spikes += 1
        else:
            did_spike = False
    spike_rate=spikes/100*100
    return V,spikes

In [11]:
def spktrain2spks(V):
    js=np.zeros(V.size)
    for i in range (V.size):
        if V[i]==80:
            js[i]=80
    return js

In [12]:
def generateSpikeRates(input):
    
    """
    Encodes input into resulting spike rates for all timesteps in a time frame given input current injected 
    Parameters:
        input: Array of inputs at timesteps t
    Returns:
        spike_rate: Array of spiking rates at timesteps t
    """
    t = 100
    dT = .01
    time = np.arange(0, t + dT, dT)
    
    spike_rate = np.empty(input.size).astype(float)
    for i in range(input.size):
        """calculate spike rate for each iteration"""

        I = input[i]*np.ones(len(time))

        # run LIF simulation
        V,spikes=LIF(I=I)

        # calculate the spike rate during the period
        spike_rate[i] = spikes / (time[len(time)-1] - time[0] ) * 100
        
    return spike_rate

def encodeInputs(inputarray):
    """
    Encodes neuron activity from x and y into rates generated from input current injected at each dT
    Parameters:
        x: Array of binary values representing x
        y: Array of binary values representing y
    Returns:
        x_out: Inputs of x encoded by their corresponding firing rates
        y_out: Inputs of y encoded by their corresponding firing rates
    """
    # example: [1 0 1 0] becomes [51 25 52 25]
    one_curr = 107
    zero_curr = 58

    inputToCurr = lambda t: zero_curr if t == 0 else (one_curr if t == 1 else -1)
    
    one_rt = 51
    zero_rt = 25
    
    out=inputarray
    out=np.multiply(out,one_curr-zero_curr)
    out=np.add(out,zero_curr)
    
    #x_in = np.array([inputToCurr(out) for xi in inputarray])
#     y_in = np.array([inputToCurr(yi) for yi in y])

    x_out=np.zeros([len(inputarray),len(inputarray[0])])
    #x_out = generateSpikeRates(x_in)
    for i in range(len(x_out)):
        x_out[i]=generateSpikeRates(out[i])
#    y_out = generateSpikeRates(y_in)

    return x_out

In [13]:
x_out=encodeInputs([[0,0], [0,1], [1,0],[1,1]])
print(len(x_out))
# np.zeros([2,4])

4


In [14]:
def train(inputs,l_rt,epoch):
    print("---------------training---------------")
    eout = encodeInputs(inputs)
    #inputs=[[0,0], [0,1], [1,0],[1,1]]
    #weights[0] = w_x,out; weights[1] = w_y,out
    weights = 10*np.ones([len(inputs[0])]).astype(float)

    l_rt = 0.0005

    for epoch in range(epoch):

        o_input = np.dot(inputs, weights) # [w_sum1, w_sum2, w_sum3, w_sum4]
        print("input currents: \n", o_input)

        o_out = generateSpikeRates(o_input) #generates spike rates from corresponding input currents calculated for o_input
        print("output firing rates: \n", o_out)

        #err = np.subtract(o_teach, o_out) #computed error between calculated spike rts and target rts (teacher)
        #print("err: ", err)
        w_adj=np.zeros(len(weights))
        for i in range(len(inputs)): #computes new weights
            for j in range(len(weights)):
                w_adj[j] = l_rt*eout[i][j]*o_out[i] #standard Hebb model modified for perceptrons/teacher neuron => Δw=a*v_pre*(target-v_post)
                weights[j] += w_adj[j] 
            print(weights)
        
    print("--------------------------------------")

    return weights

In [15]:
def predict(input, weights, thresh, one_rt=51, zero_rt=25):
    
    print("input: ", input)
    
    p_sum = np.dot(input, weights)
    p_rt = generateSpikeRates(np.array([p_sum]))
    
    #zero_lo = zero_rt - thresh
    #zero_hi = zero_rt + thresh
    
    one_lo = one_rt - thresh
    one_hi = one_rt + thresh
    prediction = 0
    if one_lo <= p_rt <= one_hi:
        prediction = 1

    print("predicted value: ", prediction)

In [85]:
x=[[0,0], [0,1], [1,0],[1,1]]
weights = train(x,0.0005,80)


---------------training---------------
input currents: 
 [ 0. 10. 10. 20.]
output firing rates: 
 [0. 0. 0. 3.]
[10. 10.]
[10. 10.]
[10. 10.]
[10.0765 10.0765]
input currents: 
 [ 0.     10.0765 10.0765 20.153 ]
output firing rates: 
 [0. 0. 0. 3.]
[10.0765 10.0765]
[10.0765 10.0765]
[10.0765 10.0765]
[10.153 10.153]
input currents: 
 [ 0.    10.153 10.153 20.306]
output firing rates: 
 [0. 0. 0. 3.]
[10.153 10.153]
[10.153 10.153]
[10.153 10.153]
[10.2295 10.2295]
input currents: 
 [ 0.     10.2295 10.2295 20.459 ]
output firing rates: 
 [0. 0. 0. 3.]
[10.2295 10.2295]
[10.2295 10.2295]
[10.2295 10.2295]
[10.306 10.306]
input currents: 
 [ 0.    10.306 10.306 20.612]
output firing rates: 
 [0. 0. 0. 3.]
[10.306 10.306]
[10.306 10.306]
[10.306 10.306]
[10.3825 10.3825]
input currents: 
 [ 0.     10.3825 10.3825 20.765 ]
output firing rates: 
 [0. 0. 0. 4.]
[10.3825 10.3825]
[10.3825 10.3825]
[10.3825 10.3825]
[10.4845 10.4845]
input currents: 
 [ 0.     10.4845 10.4845 20.969 ]
output 

input currents: 
 [ 0.     21.8535 21.8535 43.707 ]
output firing rates: 
 [ 0.  4.  4. 17.]
[21.8535 21.8535]
[21.9035 21.9555]
[22.0055 22.0055]
[22.439 22.439]
input currents: 
 [ 0.    22.439 22.439 44.878]
output firing rates: 
 [ 0.  5.  5. 18.]
[22.439 22.439]
[22.5015 22.5665]
[22.629 22.629]
[23.088 23.088]
input currents: 
 [ 0.    23.088 23.088 46.176]
output firing rates: 
 [ 0.  5.  5. 18.]
[23.088 23.088]
[23.1505 23.2155]
[23.278 23.278]
[23.737 23.737]
input currents: 
 [ 0.    23.737 23.737 47.474]
output firing rates: 
 [ 0.  6.  6. 19.]
[23.737 23.737]
[23.812 23.89 ]
[23.965 23.965]
[24.4495 24.4495]
input currents: 
 [ 0.     24.4495 24.4495 48.899 ]
output firing rates: 
 [ 0.  6.  6. 20.]
[24.4495 24.4495]
[24.5245 24.6025]
[24.6775 24.6775]
[25.1875 25.1875]
input currents: 
 [ 0.     25.1875 25.1875 50.375 ]
output firing rates: 
 [ 0.  7.  7. 21.]
[25.1875 25.1875]
[25.275 25.366]
[25.4535 25.4535]
[25.989 25.989]
input currents: 
 [ 0.    25.989 25.989 51.978

In [112]:
sample1 = np.array([0,0])
sample2 = np.array([1,0])
sample3 = np.array([0,1])
sample4 = np.array([1,1])
np.dot(sample2, weights)
predict(sample1, weights, 30)
predict(sample2, weights, 30)
predict(sample3, weights, 30)
predict(sample4, weights, 30)

input:  [0 0]
predicted value:  0
input:  [1 0]
predicted value:  1
input:  [0 1]
predicted value:  1
input:  [1 1]
predicted value:  1


In [1]:
from sklearn.datasets import load_digits
 
digits = load_digits()
print(digits.data.shape)
import matplotlib.pyplot as plt 

(1797, 64)


In [8]:
#flattening to 1D arrays
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))
#partitioning to training set, testing set, validation set
from sklearn.model_selection import train_test_split
X_train, X_other, y_train, y_other = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
X_test, X_val, y_test, y_val = train_test_split(X_other, y_other, test_size=0.5, shuffle=False)
print("Numbers to train: " + str(len(X_train)))
print("Number of train's target: " + str(len(y_train)))
print("Numbers to test: " + str(len(X_test)))
print("Number of test's target: " + str(len(y_test)))
print("Numbers to validate: " + str(len(X_val)))
print("Number of validate's target: " + str(len(y_val)))

Numbers to train: 1437
Number of train's target: 1437
Numbers to test: 180
Number of test's target: 180
Numbers to validate: 180
Number of validate's target: 180


In [18]:
weights = train(X_train[0:20],0.0005,1)

---------------training---------------
input currents: 
 [2940. 3130. 3440. 2670. 2580. 3420. 3060. 2900. 3570. 3290. 3220. 3190.
 2560. 3210. 3480. 3300. 3150. 3300. 2620. 2650.]
output firing rates: 
 [1428. 1428. 1666. 1250. 1250. 1666. 1428. 1428. 1666. 1666. 1666. 1666.
 1250. 1666. 1666. 1666. 1428. 1666. 1250. 1250.]
[ 27.85   27.85  119.242 264.898 192.784  46.414  27.85   27.85   27.85
  27.85  264.898 295.6   207.778 295.6   119.242  27.85   27.85   82.828
 295.6    64.264  27.85  226.342 172.078  27.85   27.85  101.392 247.762
  27.85   27.85  172.078 172.078  27.85   27.85  119.242 172.078  27.85
  27.85  192.784 172.078  27.85   27.85  101.392 226.342  27.85   46.414
 247.762 155.656  27.85   27.85   64.264 284.176 119.242 207.778 247.762
  27.85   27.85   27.85   27.85  137.092 264.898 207.778  27.85   27.85
  27.85 ]
[ 45.7    45.7   137.092 502.66  447.682 155.656  45.7    45.7    45.7
  45.7   282.748 511.942 517.654 478.384 137.092  45.7    45.7   100.678
 368.428 349